In [ ]:
pip install matplotlib

In [ ]:
!nvidia-smi

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Input
# import tf_slim as slim

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageChops, ImageEnhance
import os
import itertools
import random
%matplotlib inline
np.random.seed(2)

In [ ]:
def convert_to_ela_image(path, quality):
    temp_filename = 'temp_file_name.jpg'
    ela_filename = 'temp_ela.png'
    
    image = Image.open(path).convert('RGB')
    image.save(temp_filename, 'JPEG', quality = quality)
    temp_image = Image.open(temp_filename)
    
    ela_image = ImageChops.difference(image, temp_image)
    
    extrema = ela_image.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff
    
    ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)
    
    return ela_image

In [ ]:
h = 224
w = 224
image_size = (h, w)

np_arr = lambda img: np.array(img.resize(image_size)).flatten() / 255.0

def prepare_image(image_path):
    return np_arr(convert_to_ela_image(image_path, 95))

In [ ]:
X = [] # ELA converted images
Y = [] # 0 for fake, 1 for real

In [ ]:
def prepare_data(path, cls):
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            if filename.endswith('jpg') or filename.endswith('png'):
                try:
                    full_path = os.path.join(dirname, filename)
                    X.append(prepare_image(full_path))
                    Y.append(cls)
                except:
                    pass
                if len(Y) % 500 == 0:
                    print('Processing {} images'.format(len(Y)))

In [ ]:
#place authentic
Au_path = '../synthetic/Au'
prepare_data(Au_path, 1)
random.shuffle(X)
print(len(X), len(Y))

In [ ]:
#place tampered
Tp_path = '../synthetic/Tp'
prepare_data(Tp_path, 0)
print(len(X), len(Y))

In [ ]:
X = np.array(X)
Y = to_categorical(Y, 2)
X = X.reshape(-1, h, w, 3)

x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size = 0.2, random_state=5)

X = X.reshape(-1,1,1,1)

In [ ]:
input = Input(shape=(h, w, 3))
x = Conv2D(32, 3, padding='valid', activation='relu')(input)
x = Conv2D(32, 3, padding='valid', activation='relu')(x)
x = MaxPool2D(pool_size=2)(x)
x = BatchNormalization()(x)
# x = Dropout(0.25)(x)

x = Conv2D(32, 3, padding='valid', activation='relu')(x)
x = BatchNormalization()(x)
x = Conv2D(32, 3, padding='valid', activation='relu')(x)
x = MaxPool2D(pool_size=2)(x)
x = BatchNormalization()(x)
# x = Dropout(0.25)(x)

# x = Conv2D(64, 3, padding='valid', activation='relu')(x)
# x = BatchNormalization()(x)
# x = Conv2D(64, 3, padding='valid', activation='relu')(x)
# x = MaxPool2D(pool_size=2)(x)
# x = BatchNormalization()(x)

# x = Conv2D(256, 3, padding='valid', activation='relu')(x)
# x = BatchNormalization()(x)
# x = Conv2D(256, 3, padding='valid', activation='relu')(x)
# x = MaxPool2D(pool_size=2)(x)
# x = BatchNormalization()(x)

# x = Conv2D(512, 3, padding='valid', activation='relu')(x)
# x = BatchNormalization()(x)
# x = Conv2D(512, 3, padding='valid', activation='relu')(x)
# x = MaxPool2D(pool_size=2)(x)
# x = BatchNormalization()(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
# x = Dropout(0.5)(x)
output = Dense(2, activation='softmax')(x)

model = tf.keras.Model(inputs=[input], outputs=[output])
model.summary()

In [ ]:
epochs = 50
batch_size = 32
init_lr = 1e-4
# optimizer = Adam(lr = init_lr)
optimizer = Adam(lr = init_lr, decay = init_lr/epochs)

early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0,patience=3, verbose=0, mode='auto')

checkpoint_filepath = 'ela_synthetic/checkpoint'
model_checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True, monitor='val_accuracy', mode='max', save_best_only=True)

model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val), callbacks=[early_stopping, model_checkpoint_callback])

In [ ]:
X_test_set = [] # SRM converted images
Y_test_set = [] # 0 for fake, 1 for real

In [ ]:
def prepare_data(path, cls):
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            if filename.endswith('jpg') or filename.endswith('png'):
                try:
                    full_path = os.path.join(dirname, filename)
                    X_test_set.append(prepare_image(full_path))
                    Y_test_set.append(cls)
                except:
                    pass
                if len(Y_test_set) % 500 == 0:
                    print('Processing {} images'.format(len(Y_test_set)))

In [ ]:
#place authentic
# synthetic_test/Au
Au_path = '../synthetic_test/Au'
prepare_data(Au_path, 1)
random.shuffle(X_test_set)
# X = X[:2100]
# Y = Y[:2100]
print(len(X_test_set), len(Y_test_set))

In [ ]:
#place tampered
Tp_path = '../synthetic_test/Tp'
prepare_data(Tp_path, 0)
print(len(X_test_set), len(Y_test_set))

In [ ]:
X_test_set = np.array(X_test_set)
Y_test_set = to_categorical(Y_test_set, 2)
X_test_set = X_test_set.reshape(-1, h, w, 3)

x_test, x_test2, y_test, y_test2 = train_test_split(X_test_set, Y_test_set, test_size = 0.2, random_state=5)

# X_test = X_test.reshape(-1,1,1,1)

In [ ]:
from sklearn.metrics import average_precision_score

predictions = model.predict(x_test)
average_precision = average_precision_score(y_test, predictions)

In [ ]:
average_precision

In [ ]:
plt.plot(history.history['accuracy'], color='b', label="Training accuracy")
plt.plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
legend = plt.legend(loc='best', shadow=True)
plt.savefig("acc.png", bbox_inches='tight', pad_inches=0)

In [ ]:
from tensorflow.keras.models import load_model

model_1 = model.load_weights('srm_synthetic_7/checkpoint.hdf5')

In [ ]:
predictions = model_1.predict(x_test)
average_precision = average_precision_score(y_test, predictions)